# Lab 7 :  Graphical and hidden Markov models
```
- [S23] Advanced Machine Learning, Innopolis University
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
1. Project start
2. Recap on HMMs
3. Simple example for HMMs
3. Task
```

<hr>


## 1. Recap on HMMs and Likelihood

<!--![](https://www.dropbox.com/scl/fi/wm8nbmqgchv3orw7r488f/Lecture-5-HMM.jpg?rlkey=3x0lmmzxchgz6esmq6yrjfzne&dl=1)-->

In [1]:
import numpy as np

states = ('Sunny', 'Rainy')
observations = ('happy', 'grumpy')
pi = np.array([2. / 3., 1. / 3.])  # initial probability
A = np.array([[7. / 9., 2. / 9.], [0.4, 0.6]])  # Transmission probability
B = np.array([[0.8, 0.2], [0.4, 0.6]])  # Emission probability
bob_says = np.array([0, 0, 1, 1, 1, 0])


def forward(obs_seq, pi, A, B):
    T = len(obs_seq)
    N = A.shape[0]
    alpha = np.zeros((T, N))
    alpha[0] = pi * B[:, obs_seq[0]]
    for t in range(1, T):
        alpha[t] = np.inner(alpha[t - 1], A) * B[:, obs_seq[t]]
    return alpha


def likelihood(alpha):
    # using the forward part of the forward-backward algorithm
    return alpha[-1].sum()


alpha = forward(bob_says, pi, A, B)
print(alpha)

print(likelihood(alpha))

[[0.53333333 0.13333333]
 [0.35555556 0.11733333]
 [0.06052346 0.12757333]
 [0.01508469 0.06045203]
 [0.00503326 0.02538306]
 [0.00764435 0.00689726]]
0.014541607035957256


## 2. Viterbi algorithm simple example


In [2]:
from numpy import random

# Transition Probabilities (s - sunny; r-raining)
p_ss = 7. / 9.
p_sr = 2. / 9.
p_rs = 0.4
p_rr = 0.6

# Initial Probabilities
p_s = 2 / 3
p_r = 1 / 3

# Emission Probabilities (h - happy; g- grumpy)
p_sh = 0.8
p_sg = 0.2
p_rh = 0.4
p_rg = 0.6

moods = ['H', 'H', 'G', 'G', 'G', 'H']
probabilities = []
weather = []

if moods[0] == 'H':
    probabilities.append((p_s * p_sh, p_r * p_rh))
else:
    probabilities.append((p_s * p_sg, p_r * p_rg))

for i in range(1, len(moods)):
    yesterday_sunny, yesterday_rainy = probabilities[-1]
    if moods[i] == 'H':
        today_sunny = max(yesterday_sunny * p_ss * p_sh, yesterday_rainy * p_rs * p_sh)
        today_rainy = max(yesterday_sunny * p_sr * p_rh, yesterday_rainy * p_rr * p_rh)
        probabilities.append((today_sunny, today_rainy))
    else:
        today_sunny = max(yesterday_sunny * p_ss * p_sg, yesterday_rainy * p_rs * p_sg)
        today_rainy = max(yesterday_sunny * p_sr * p_rg, yesterday_rainy * p_rr * p_rg)
        probabilities.append((today_sunny, today_rainy))

for p in probabilities:
    if p[0] > p[1]:
        weather.append('S')
    else:
        weather.append('R')

weather

['S', 'S', 'S', 'R', 'R', 'S']

## 3. Parts of Speech (POS) Tagging

---

* process of assigning a part-of-speech to each word in a text
* POS is a disambiguation task (some words can have multiple part of speech depending in the context)
* Gives an idea about syntactic structure (parsing)

**How does it work?**

<!--![](http://www.cs.virginia.edu/~hw5x/Course/TextMining-2019Spring/_site/docs/codes/HMM.PNG)-->

## 4. Task (Viterbi algorithm)

<!-- ![](https://miro.medium.com/max/1080/1*8-5KZVj-_jZOWN83gGhD5A.png) -->
<!--![](https://www.dropbox.com/scl/fi/twg8yqdhqwxcbr5lcqj4w/pos_tag1.png?rlkey=5kizqwfv8gk8vg26434sp0rk2&dl=1)-->

First steps :
*  Calculate or estimate transition probabilities between different parts of speech tags
* Calculate or estimate prior probabilities of tags


In the task you are going to use a a fraction of conll2000 dataset. The dataset should be retrieved from : [Train](https://www.dropbox.com/s/x9n6f9o9jl7pno8/train_pos.txt?dl=1), [Test](https://www.dropbox.com/s/v8nccvq7jewcl8s/test_pos.txt?dl=1)

1. Calculate the transition probability and emission matrices (First step towards viterbi) <!-- 10 points -->
1. Use Viterbi algorithm for POS tagging task (you can use libiries such as [`hmmlearn`](https://github.com/hmmlearn/hmmlearn)).
1. Test your viterbi model on the test set and record the accuracy. The accuray referes to the number of correcly predicted tags in the whole test samples.
1. Using Recurrent neural network (RNN, GRU or LSTM) or Transfomer solve the task for POS

In [ ]:
# Load dataset train and test
from datasets import load_dataset

dataset = load_dataset('conll2000')

train_data = dataset['train']
test_data = dataset['test']

print(len(train_data))

In [13]:
print(train_data[0]['pos_tags'])

[19, 14, 11, 19, 39, 27, 37, 32, 34, 11, 15, 19, 14, 19, 22, 14, 20, 5, 15, 14, 19, 19, 5, 34, 32, 34, 11, 15, 19, 14, 20, 9, 20, 24, 15, 22, 6]


    - name: pos_tags
      sequence:
        class_label:
          names:
            '0': ''''''
            '1': '#'
            '2': $
            '3': (
            '4': )
            '5': ','
            '6': .
            '7': ':'
            '8': '``'
            '9': CC
            '10': CD
            '11': DT
            '12': EX
            '13': FW
            '14': IN
            '15': JJ
            '16': JJR
            '17': JJS
            '18': MD
            '19': NN
            '20': NNP
            '21': NNPS
            '22': NNS
            '23': PDT
            '24': POS
            '25': PRP
            '26': PRP$
            '27': RB
            '28': RBR
            '29': RBS
            '30': RP
            '31': SYM
            '32': TO
            '33': UH
            '34': VB
            '35': VBD
            '36': VBG
            '37': VBN
            '38': VBP
            '39': VBZ
            '40': WDT
            '41': WP
            '42': WP$
            '43': WRB
    - name: chunk_tags
      sequence:
        class_label:
          names:
            '0': O
            '1': B-ADJP
            '2': I-ADJP
            '3': B-ADVP
            '4': I-ADVP
            '5': B-CONJP
            '6': I-CONJP
            '7': B-INTJ
            '8': I-INTJ
            '9': B-LST
            '10': I-LST
            '11': B-NP
            '12': I-NP
            '13': B-PP
            '14': I-PP
            '15': B-PRT
            '16': I-PRT
            '17': B-SBAR
            '18': I-SBAR
            '19': B-UCP
            '20': I-UCP
            '21': B-VP
            '22': I-VP

In [79]:
print(train_data.shape[0])

8937


In [45]:
from collections import defaultdict

# Calculate transition probabilities between different part of speach tags
def calculate_transition_probability(data):
    # Dictionary for count tags
    tag_counts = defaultdict(int)
    # Dictionary for count transition between every POS
    transition_counts = defaultdict(lambda: defaultdict(int))

    for features in data:
        # Length sentence
        size = len(features['pos_tags'])

        for index, pos_tag in enumerate(features['pos_tags']):
            if index < size - 1:
                next_pos_tag = features['pos_tags'][index + 1]

                # Calculate count tags in sentences
                tag_counts[pos_tag] += 1

                # Calculate count transition between POS_t and POS_{t+1}
                transition_counts[pos_tag][next_pos_tag] += 1

    # Dictionary for probability transition
    transition_probability = defaultdict(lambda: defaultdict(int))

    # Calculate transition probabilities for every transition pos
    for tag in transition_counts:
        for next_tag in transition_counts[tag]:
            transition_probability[tag] = transition_counts[tag][next_tag] / tag_counts[next_tag]
    return transition_probability

In [46]:
calculate_transition_probability(train_data)

defaultdict(<function __main__.calculate_transition_probability.<locals>.<lambda>()>,
            {19: 0.024096385542168676,
             14: 0.004016064257028112,
             11: 0.26666666666666666,
             39: 0.0036496350364963502,
             27: 0.02631578947368421,
             37: 0.0024096385542168677,
             32: 0.003115264797507788,
             34: 0.00267379679144385,
             15: 0.01818181818181818,
             22: 0.0053475935828877,
             20: 0.001594896331738437,
             5: 0.001049317943336831,
             9: 0.012048192771084338,
             24: 0.004016064257028112,
             38: 0.0024096385542168677,
             36: 0.0003486750348675035,
             26: 0.00018615040953090097,
             10: 0.000531632110579479,
             8: 0.003115264797507788,
             0: 0.0005714285714285715,
             35: 0.0013844023996308261,
             12: 0.00019688915140775743,
             18: 0.004016064257028112,
             1: 0

In [89]:
# Calculate prior probabilities
def calculate_prior_probabilities(data):
    # Dictionary count every POS start in sentence
    start_tag_counts = defaultdict(int)
s
    # Count total sentences
    total_sentences = 0
    
    for features in data:
        if features['pos_tags']:
            start_tag = features['pos_tags'][0]
            start_tag_counts[start_tag] += 1
            total_sentences += 1
    
    # Dictionary prior probabilities for every POS
    prior_probability = defaultdict(int)
    
    for tag in start_tag_counts:
        prior_probability[tag] = start_tag_counts[tag] / total_sentences
        
    return prior_probability

In [91]:
calculate_prior_probabilities(train_data)

defaultdict(int,
            {19: 0.045210384959713516,
             20: 0.19192032229185318,
             9: 0.056177260519247985,
             11: 0.21239928379588183,
             8: 0.07273948075201432,
             22: 0.04364368845120859,
             14: 0.12891674127126232,
             25: 0.0583034914950761,
             27: 0.06031781557743957,
             37: 0.005259623992837959,
             15: 0.044874664279319604,
             26: 0.00861683079677708,
             43: 0.006378692927484333,
             34: 0.004811996418979409,
             41: 0.0032452999104744854,
             10: 0.016226499552372427,
             17: 0.004923903312444047,
             32: 0.0033572068039391225,
             36: 0.01342882721575649,
             3: 0.003133393017009848,
             16: 0.0011190689346463742,
             0: 0.0004476275738585497,
             28: 0.002126230975828111,
             35: 0.0011190689346463742,
             12: 0.0032452999104744854,
             39: